# **Training, Testing and Validating the Model**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [2]:
from google.colab import files
import io

uploaded = files.upload()
for file_name in uploaded.keys():
    raw_data = pd.read_csv(io.BytesIO(uploaded[file_name]), delimiter=';')

Saving Raw Data.csv to Raw Data.csv


In [3]:
data = pd.read_csv('/content/Raw Data.csv', delimiter=';')
data.head(3)

,Comment,Emotion
0,a boyfriend with whom i split up with came ove...,anger
1,a certain friend tried to push me off a seat i...,anger
2,a father of children killed in an accident,sadness


The model

In [4]:
# Step 1: Load and Prepare Your Data

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(data['Comment'], data['Emotion'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_data(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors='tf')

train_encodings = tokenize_data(X_train)
val_encodings = tokenize_data(X_val)
test_encodings = tokenize_data(X_test)

# Step 3: Convert Labels to TensorFlow Format
label_map = {label: idx for idx, label in enumerate(y_train.unique())}
y_train_numeric = y_train.map(label_map)
y_val_numeric = y_val.map(label_map)
y_test_numeric = y_test.map(label_map)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train_numeric)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), y_val_numeric)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test_numeric)).batch(32)

# Step 4: Model Training
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=2)

# Step 5: Evaluate the Model
y_pred = model.predict(test_dataset)
y_pred_labels = tf.argmax(y_pred.logits, axis=1)

# Calculate accuracy and classification report
print(f'Test Accuracy: {accuracy_score(y_test_numeric, y_pred_labels)}')
print(classification_report(y_test_numeric, y_pred_labels, target_names=label_map.keys()))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
438/438 [==============================] - 307s 505ms/step - loss: 0.4756 - accuracy: 0.8344 - val_loss: 0.1636 - val_accuracy: 0.9353
Epoch 2/2
94/94 [==============================] - 25s 184ms/step
Test Accuracy: 0.933
              precision    recall  f1-score   support

       anger       0.92      0.92      0.92       385
         joy       0.94      0.96      0.95      1017
        fear       0.82      0.99      0.90       338
     sadness       0.98      0.95      0.97       878
     anxious       0.90      0.75      0.82       113
        love       0.93      0.80      0.86       269

    accuracy                           0.93      3000
   macro avg       0.92      0.89      0.90      3000
weighted avg       0.94      0.93      0.93      3000



Save and dowload the model

In [5]:
model.save_pretrained('/content/bert_model')
tokenizer.save_pretrained('/content/bert_model')

# Download the saved model files
import shutil

# Zip the model directory
shutil.make_archive('/content/bert_model', 'zip', '/content/bert_model')

# Download the zip file
from google.colab import files
files.download('/content/bert_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Using the Model on New Data**

Import libraries

In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from google.colab import files
import zipfile
import os,io

Extract the Zip File

In [ ]:
# Upload the zip file
uploaded = files.upload()

# Assuming the zip file is named 'bert_model.zip'
with zipfile.ZipFile('Bert_Model.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Bert_Model')


Load the Model and Tokenizer

In [ ]:
# Define the path to the extracted model directory
model_dir = '/content/Bert_Model'

# Load the model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir)


Load the dataset

In [ ]:
uploaded = files.upload()
for file_name in uploaded.keys():
    data = pd.read_csv(io.BytesIO(uploaded[file_name]), delimiter=';')
    data.head(3)

comments = data['Comment']  # Adjust this to match your column name

Tokenize the Comments

In [ ]:
inputs = tokenizer(comments.tolist(), padding=True, truncation=True, return_tensors='pt')

Make Predictions

In [ ]:
# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # Get the raw predictions

# Calculate probabilities
probabilities = torch.softmax(logits, dim=1)  # Convert logits to probabilities

Interpret Predictions

In [ ]:
# Get the predicted class
predicted_classes = torch.argmax(probabilities, dim=1)

# Assuming you have a mapping of class indices to emotions
emotion_mapping = {0: 'Emotion1', 1: 'Emotion2', 2: 'Emotion3'}  # Update with your actual mappings
predicted_emotions = [emotion_mapping[idx.item()] for idx in predicted_classes]

# Combine results into a DataFrame
results = pd.DataFrame({
    'Comment': comments,
    'Predicted Emotion': predicted_emotions,
    'Probability': [prob.max().item() for prob in probabilities]
})

# Display results
print(results)

Save in csv format

In [ ]:
# Save results to a CSV file
results.to_csv('predicted_emotions.csv', index=False)

# Download the CSV file
from google.colab import files
files.download('predicted_emotions.csv')